In [1]:
import numpy as np
import networkx as nx
import powerlaw 
import netgraph
import matplotlib.pylab as plt
from matplotlib.pyplot import savefig
import pandas as pd
import os
import pickle

from Qommunity.samplers.hierarchical.advantage_sampler import AdvantageSampler
from Qommunity.samplers.regular.leiden_sampler import LeidenSampler
from Qommunity.samplers.regular.louvain_sampler import LouvainSampler
#from Qommunity.samplers.regular.bayan_sampler import BayanSampler

from Qommunity.searchers.community_searcher.community_searcher import CommunitySearcher
from Qommunity.searchers.hierarchical_community_searcher import HierarchicalCommunitySearcher

from iterative_searcher.iterative_searcher import IterativeSearcher

In [2]:
class GraphFromCSV:
    """
    It creates an object with different properties from a csv. Everything related to it, will be
        saved with the provided 'name' followed by the proper extensions.
    As of now, this is not the latest version of this class (or python script), but for this pipeline is enough.
    """

    def __init__(self, graph, name, base_dir='/'):
        self.graph = graph  # Graph file name
        self.conns = pd.read_csv(graph, delimiter=',', header=None).values # Graph connections
        self.graph_size = self.conns.shape
            
        self.name = name
        if base_dir == '/':
            self.dir = os.getcwd()+'/'
        else:
            self.dir = base_dir
        self.originals = self.conns
        # TODO: Add check path for base_dir!

    def __revert(self):
        """
        Reorders AAL3 regions by hemispheres.
        Odd indices correspond to Left hemisphere regions.
        Even indices correspond to rigth hemisphere regions.
        Stores a dictionary with the reodering of indices.
        """
        odd_odd = self.conns[::2, ::2]
        odd_even = self.conns[::2, 1::2]
        first = np.vstack((odd_odd, odd_even))
        even_odd = self.conns[1::2, ::2]
        even_even= self.conns[1::2, 1::2]
        second = np.vstack((even_odd, even_even))
        self.conns = np.hstack((first,second))

        # To map actual labels with original ones
        labels = np.array([x for x in range(0, self.graph_size[0])])
        left = np.array([x for x in range(1, self.graph_size[0], 2)])
        rigth = np.array([x for x in range(0, self.graph_size[0], 2)])
        self.hemis = dict(zip(labels, np.concatenate((left, rigth), axis=0)))

    def __take_log(self):
        """
        Takes the natural logarithm of the connections. Enhances visualisation of the matrix.
        """
        self.conns = np.log1p(self.conns)

    def __plot_graph(self, save=True, show=False, fig_size=(20,15), dpi=500):
        """
        Plot a graph. It assumes that the adjancency matrix is a csv file.
        """
        plt.figure(figsize=fig_size)
        plt.imshow(self.conns)
        cbar = plt.colorbar()
        cbar.set_label('Connection Strength', rotation=270)
        plt.tight_layout()
        if save:
            plt.savefig(self.dir+self.name+'.svg', format='svg', dpi=dpi)      
        if show:
            plt.show()     

    def process_graph(self, log=True, reshuffle=True, save=True, show=False, fig_size=(20,15)):
        """
        Applies default operations to the graph to work with it.
        """
        self.processed = True # The object has been processed
        if self.conns.shape[0] <= 1:
            raise ValueError("You are trying to process a flat graph. Can't do it. Unflatten your graph and set it to default.")
        else:
            if log:
                self.__take_log()
            if reshuffle:
                self.__revert()
            self.__plot_graph(save=save, show=show, fig_size=(20,15))
    
    def get_connections(self, ini=False):
        if not ini:
            return self.conns 
        else:
            return self.originals
    
    def flatten_graph(self, save=True):
        """
        Flatten the lower triangular adjancency matrix of the graph. 
        The flattened graph becomes available after applying this method.
        """
        x = self.conns.shape[0] # Dimensions of the graph 
        if x <= 1:
            raise ValueError("Dimension of the graph is 1 (or lower). You can't flattened an already flattened graph")
        else:
            dims = int(self.conns.shape[0]*(self.conns.shape[0]-1)/2)
            self.flat_conns = np.zeros((1,dims))
            k = 0
            for i in range(x):
                for j in range(i):
                    self.flat_conns[0,k] = self.conns[i,j]
                    k += 1
            if save:
                np.savetxt(self.dir+self.name+'_flatCM.csv', self.flat_conns, delimiter=',')
            return self.flat_conns

    def unflatten_graph(self, to_default=False):
        """
        Unflatten a graph and transform it to a square symmetric matrix. 
        The unflattened graph becomes available after applying this method.
        to_default: bool - The unflattened matrix becomes the default graph and replaces 
            the initial flat graph. As a checkpoint, the flattened graph is saved in the directory(default: False)
        """
        x = self.conns.shape[0] # First dimension of the flattened graph 
        flat_dim = self.conns.shape[1]
        if x > 1:
            raise ValueError("Dimension of the graph greater than 1. You can't unflattened an already unflattened graph")
        else:
            dims = int(1+np.sqrt(1+8*flat_dim)/2) # Dimensions of the squared graph
            self.unflat_conns = np.zeros((dims,dims))
            k = 0
            for i in range(dims):
                for j in range(i):
                    self.unflat_conns[i, j] = self.conns[0, k]
                    self.unflat_conns[j, i] = self.conns[0, k]
                    k += 1
            if to_default:
                # We save the flat graph with another name
                np.savetxt(self.dir+self.name+'_flatCM.csv', self.conns, delimiter=',')
                # We replace the original file with the unflattend graph
                np.savetxt(self.graph, self.unflat_conns, delimiter=',')
                # We re-initialize the graph with the unflattened graph and both the same name and directory
                self.__init__(self.graph, self.name, self.dir)
            return self.unflat_conns

In [3]:
G_process = GraphFromCSV("Brain-networks/control/ses-preop/sub-CON10_ses-preop_flatCM.csv", name="sub-CON10_ses-preop", base_dir="Brain-networks/")
network = G_process.unflatten_graph(to_default=False)
network = np.delete(network, [34,35,80,81], axis=0)
network = np.delete(network, [34,35,80,81], axis=1)
graph = nx.from_numpy_array(network, create_using=nx.Graph, edge_attr="weight")

In [4]:
num_runs = 20
resolution = 1
 
_, ms_Louv, _ = IterativeSearcher(LouvainSampler(graph, resolution=resolution)).run(num_runs=num_runs, save_results=False)
print("Louvain modularity:", ms_Louv.max())
_, ms_Leid, _ = IterativeSearcher(LeidenSampler(graph, resolution=resolution)).run(num_runs=num_runs, save_results=False)
print("Leiden modularity:", ms_Leid.max())

100%|██████████| 20/20 [00:01<00:00, 19.87it/s]


Louvain modularity: 0.6109030329878278


100%|██████████| 20/20 [00:00<00:00, 76.21it/s]

Leiden modularity: 0.6112827841745766


In [5]:
adv_searcher = IterativeSearcher(AdvantageSampler(graph, resolution=resolution, num_reads=100, use_clique_embedding=True))
results_adv = adv_searcher.run_with_sampleset_info(num_runs=num_runs, save_results=False)

100%|██████████| 20/20 [40:53<00:00, 122.69s/it]


In [14]:
# np.save("adv-brain_network-runs_20.npy", results_adv)
# Moved the file to a new folder
# np.load("/Brain-networks-results/adv-brain_network-runs_20.npy",allow_pickle=True)

In [6]:
ms_Adv = np.array([results_adv[i][1] for i in range(num_runs)])
communities, division_tree, division_modularities = results_adv[ms_Adv.argmax()][0], results_adv[ms_Adv.argmax()][3], results_adv[ms_Adv.argmax()][4]

In [125]:
communities = results_adv.communities
modularities = results_adv.modularity
division_trees = results_adv.division_tree
division_modularities = results_adv.division_modularities
times = results_adv.time

In [30]:
def calc_modularity_increments(division_modularities: list[float]) -> list[float]:
    mod_increments = [
        division_modularities[i + 1] - division_modularities[i]
        for i in range(0, len(division_modularities) - 1)
    ]
    return mod_increments

In [61]:
modularity_increments = []
for div_mod in division_modularities:
    mod_increments = calc_modularity_increments(div_mod)
    modularity_increments.append(np.array(mod_increments))

modularity_increments = np.array(modularity_increments)

In [87]:
[row for row in modularity_increments if np.any(row < 0)]

[array([ 4.15150293e-01,  1.53329831e-01,  1.63627253e-02,  1.10710208e-03,
        -1.77781720e-04, -3.22024096e-04,  4.70025893e-08]),
 array([ 4.16131021e-01,  1.57058913e-01,  1.47968473e-02,  2.94989164e-04,
        -1.22845798e-03, -3.54195073e-04, -1.08286304e-03]),
 array([ 4.18061073e-01,  1.55299385e-01,  1.62844319e-02,  7.66439524e-04,
         2.51151918e-04, -6.98934959e-05]),
 array([ 4.23160822e-01,  1.53718547e-01,  1.49613938e-02,  1.26441015e-03,
         1.48706309e-05, -2.20727940e-03]),
 array([ 0.42000781,  0.1582157 ,  0.01579647,  0.00103627, -0.00176747,
        -0.00057639, -0.00043425]),
 array([ 4.19562775e-01,  1.56391689e-01,  1.52191817e-02, -1.72228914e-03,
         3.54167623e-04, -5.74661750e-04, -2.90826735e-04]),
 array([ 4.20766689e-01,  1.57415803e-01,  1.46004669e-02, -9.02911115e-04,
        -6.94000455e-04, -5.61437220e-06,  4.50274390e-08]),
 array([ 4.17447328e-01,  1.59257661e-01,  1.43350310e-02,  8.29910913e-04,
         1.20550957e-04,  6

In [90]:
np.vectorize(lambda row: (row < 0).any())(modularity_increments)

array([False, False, False,  True,  True, False, False, False, False,
        True,  True,  True,  True,  True,  True,  True, False, False,
        True,  True])

In [91]:
has_neg_mod_increase = np.vectorize(lambda row: (row < 0).any())

In [116]:
has_neg_mod_increase(modularity_increments)

array([False, False, False,  True,  True, False, False, False, False,
        True,  True,  True,  True,  True,  True,  True, False, False,
        True,  True])

In [115]:
modularity_increments[np.where(has_neg_mod_increase(modularity_increments))]

array([array([ 4.15150293e-01,  1.53329831e-01,  1.63627253e-02,  1.10710208e-03,
              -1.77781720e-04, -3.22024096e-04,  4.70025893e-08])                ,
       array([ 4.16131021e-01,  1.57058913e-01,  1.47968473e-02,  2.94989164e-04,
              -1.22845798e-03, -3.54195073e-04, -1.08286304e-03])                ,
       array([ 4.18061073e-01,  1.55299385e-01,  1.62844319e-02,  7.66439524e-04,
               2.51151918e-04, -6.98934959e-05])                                 ,
       array([ 4.23160822e-01,  1.53718547e-01,  1.49613938e-02,  1.26441015e-03,
               1.48706309e-05, -2.20727940e-03])                                 ,
       array([ 0.42000781,  0.1582157 ,  0.01579647,  0.00103627, -0.00176747,
              -0.00057639, -0.00043425])                                      ,
       array([ 4.19562775e-01,  1.56391689e-01,  1.52191817e-02, -1.72228914e-03,
               3.54167623e-04, -5.74661750e-04, -2.90826735e-04])                ,
       array([ 4

### % Of negative modlarity increment obtained

In [123]:
modularity_increments[has_neg_mod_increase(modularity_increments)].shape[0] / modularity_increments.shape[0] * 100

55.00000000000001

### Basics

Optimum

In [128]:
ms_Adv = np.array([results_adv[i][1] for i in range(num_runs)])
communities, division_tree, division_modularities = results_adv[ms_Adv.argmax()][0], results_adv[ms_Adv.argmax()][3], results_adv[ms_Adv.argmax()][4]

In [133]:
print("Optimum:")
print(f"communities detected no.: {len(communities)}")
print(f"Modularity: {division_modularities[-1]}")
print(f"No. of recursive divisions (division levels): {len(division_tree)}")

Optimum:
communities detected no.: 21
Modularity: 0.5998335088559781
No. of recursive divisions (division levels): 7


And where negative modularity increase occured:

(suboptimas, one might say)

In [136]:
idxs = np.where(has_neg_mod_increase(modularity_increments))
idxs

(array([ 3,  4,  9, 10, 11, 12, 13, 14, 15, 18, 19], dtype=int64),)

In [161]:
comms_sub = results_adv.communities[idxs]

In [176]:
print("Where dicrease in modularity occured")
no_communities_detected_sub = [len(comm) for comm in comms_sub]
print(no_communities_detected_sub)
print(f"mean of communities det. where modularity dicrease occured: {np.array(no_communities_detected_sub).mean()}")

Where dicrease in modularity occured
[26, 32, 27, 29, 25, 28, 32, 26, 27, 26, 24]
mean of communities det. where modularity dicrease occured: 27.454545454545453


In [160]:
comms_complement = results_adv.communities[np.where(~has_neg_mod_increase(modularity_increments))]

In [175]:
print("'Standard'/'expected' behaviour - where dicrease in modularity did not occur")
no_communities_detected = [len(comm) for comm in comms_complement]
print(no_communities_detected)
print(f"mean of communities detected: {np.array(no_communities_detected).mean()}")

'Standard'/'expected' behaviour - where dicrease in modularity did not occur
[24, 27, 24, 21, 21, 28, 30, 28, 28]
mean of communities detected: 25.666666666666668
